## Author

This notebook was authored by Amogh Mannekote (UFID 27146587).

## Index of questions (for referencing in the notebook)

1. Implement the elastic net TSP by minimizing the following objective function. \
 $E_{TSP}(P, Y) = \sum_{i = 1}^{N}{\sum_{a = 1}^M{P_{ia}||x_i - y_a||^2}} + \frac{\kappa}{2}\sum_{a = 1}^M{||y_a - y_{a \oplus 1}||_2^2}$
2. Document the settings of the free parameters $\kappa$, $\beta$ (the inverse temperature) and $M$, the number of hidden and actual cities.
3. If reducing $\kappa$, provide the rate, the total number of iterations, etc.
4. If steadily increasing $\beta$, provide the rate, the total number of iterations, etc.
5. Document initial conditions on $Y$.
6. Execute the elastic net on 100 cities in $[0,1]^2$ chosen using a uniform distribution.
7. Since we require average performance, please compute the average tour length for 100 trials and show a histogram of tour lengths.
8. Show visual examples of the shortest, longest and median length tours for the 100 city TSP.
9. [Extra Credit: 30 points] Compare against well known asymptotic lower bounds such as the ones uploaded on Canvas and the Held-Karp lower bound
10. [Extra Credit: 50 points] Execute the elastic net on the US 48 cities using data from TSPlib or from https://people.sc.fsu.edu/~jburkardt/datasets/tsp/tsp.html. Give a detailed explanation
of the results
11. [Extra Credit: 50 points] Explain the equivalent objective function (free energy) shown below obtained by summing over all configurations of P
12. [Extra Credit: 70 points] Explain the relevance of the following objective function in the elastic net implementation
13. [Extra Credit: 50 points] Show the convergence of the algorithm by detailed plots of either (1) or (2) (depending on your implementation).

------------

## Imports

In [101]:
from itertools import chain, tee
from collections import deque
import math
import functools

import numpy as np
np.random.seed(42)

import wandb

from data_loader import load_cities

## Initialize hyperparameters

**NOTE TO TA: The cell below is relevant for answering questions [2, 3, 4, 5].**

In [102]:
# Log current hyperparameter settings to W&B.
sweep_config = {
    "name" : "sweep-tsp-22",
    "method" : "random",
    "parameters" : {
        "INPUT": {
            "values": ["tsp-22", "us-cities"]
        },
        "M_FACTOR" : {
            "values" : [1.5, 2.0, 2.5, 3.0, 3.5]
        },
        "KAPPA_START" : {
            "min": 0.05,
            "max": 0.5
        },
        "GAMMA": {
            "min": 1.01,
            "max": 1.25
        },
        "BETA": {
            "min": 1,
            "max": 20
        },
        "NUM_ITERS": {
            "value": 10000
        },
        "WORST_DISTANCE_EPSILON": {
            "value": 0.01
        },
        "PERTURBATION_RADIUS": {
            "min": 0.20,
            "max": 2.00
        },
        
    }
}
sweep_id = wandb.sweep(sweep_config)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: 8u3d01ax
Sweep URL: https://wandb.ai/msamogh/uncategorized/sweeps/8u3d01ax


### Initialize W&B environment

In [103]:
wandb.init(project="mis-tsp", entity="msamogh")
CONFIG = wandb.config

wandb: wandb version 0.12.14 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [104]:
def init_intermediate_cities(self, strategy="centroid", **kwargs):
    if strategy == "centroid":
        self.intermediate_cities = np.tile(
            np.mean(self.cities, axis=0),
            self.M
        ).reshape((self.M, 2))
        # Create perturbation vectors of magnitude radius
        perturbation_vectors = np.random.random((self.M, 2))
        perturbation_vectors /= np.linalg.norm(perturbation_vectors, axis=1, ord=2, keepdims=True)
        perturbation_vectors *= self.CONFIG["PERTURBATION_RADIUS"]
        self.intermediate_cities += perturbation_vectors
    else:
        raise RuntimeError(f"Unknown strategy: {strategy}")

$$\kappa_{T+1} = \frac{\kappa_T}{\gamma}$$

In [105]:
def update_kappa(self):
    self.kappa = max(self.kappa / self.CONFIG["GAMMA"], 0.01)

$$P_{ia} = \frac{exp\{-\beta\||x_i - y_a||^2_2\}}{\sum_{b=1}^{M}{exp\{-\beta||x_i - y_b||_2^2\}}}$$

In [106]:
def update_weights(self):
    self.diff = self.cities[:, np.newaxis] - self.intermediate_cities
    self.dist_squared = np.sum(self.diff ** 2, axis=-1)

    self.worst_dist = np.max(np.min(np.sqrt(self.dist_squared), axis=1))
    self.worst_dists.append(self.worst_dist)

    self.weights = np.exp(-self.dist_squared / (2 * (self.kappa ** 2)))
    self.weights /= self.weights.sum(axis=1)[:, np.newaxis]

$$d_a = \sum_{i = 1}^{N}{P_{ia}}$$

$$D_{aa} = d_a$$

In [107]:
@property
def D(self):
    def d_ij(i, j):            
        if i == j:
            return np.sum(self.weights[:, int(j)])
        return 0
    return np.fromfunction(np.vectorize(d_ij), (self.M, self.M))

$$L_{aa} = 2, L_{a, a \bigoplus 1} = -1, L_{a, a \ominus 1} = -1$$

In [108]:
@property
def L(self):
    def L_ij(i, j):
        if i == j:
            return 2
        if i == (j + 1) % self.M:
            return 1
        if j == (i + 1) % self.M:
            return 1
        return 0
    return np.fromfunction(np.vectorize(L_ij), (self.M, self.M), dtype=int)

$$Y = (\kappa L + D)^{-1}P^TX$$

In [109]:
def update_intermediate_cities(self):
    self.intermediate_cities = np.linalg.inv((self.kappa * self.L + self.D)) @ self.weights.T @ self.cities

In [110]:
def get_city_to_intermediate_city_map(self):
    hidden_visible_city_pairs = []

    for _ in range(self.dist_squared.shape[0]):
        # Identify the shortest edge among all edges between hidden and real cities.
        city, intermediate_city = np.unravel_index(np.argmin(self.dist_squared), self.dist_squared.shape)

        # Set distance from the (city, intermediate_city) pair to infinity so that
        # they don't get picked once again as the shortest edge in the subsequent
        # iterations.
        self.dist_squared[city, :] = np.inf
        self.dist_squared[:, intermediate_city] = np.inf

        hidden_visible_city_pairs.append((intermediate_city, city))

    self.city_tour = [x[1] for x in sorted(hidden_visible_city_pairs, key=lambda x: x[0])]

In [111]:
def get_tour(self):
    assert len(self.city_tour) == self.N
    distance = 0
    tour = []
    for stop_idx in range(len(self.city_tour)):
        tour.append((self.city_tour[stop_idx], self.city_tour[(stop_idx + 1) % self.N]))
        distance += np.linalg.norm(self.original_cities[stop_idx] - self.original_cities[(stop_idx + 1) % self.N])
    return distance, tour

In [112]:
def solve(self):
    for i in range(NUM_ITERS):
        self.update_weights()
        self.update_kappa()
        self.update_intermediate_cities()
        if i % 100 == 0:
            print(f"Iteration {i}, Worst distance: {self.worst_dist}")
        if self.worst_dist < self.CONFIG["WORST_DISTANCE_EPSILON"]:
            break
        if len(self.worst_dists) == 2:
            if self.worst_dists[1] - self.worst_dists[0] == 0:
                break
    self.get_city_to_intermediate_city_map()
    return self.get_tour()

In [113]:
class ElasticNet(object):
    
    def __init__(
        self,
        init_intermediate_cities,
        solve,
        update_weights,
        update_kappa,
        update_intermediate_cities,
        D,
        L,
        get_city_to_intermediate_city_map,
        get_tour
    ):
        # Take the methods as parameters to the constructor to be able to
        # split the method definitions into separate cells in the notebook.
        ElasticNet.init_intermediate_cities = init_intermediate_cities
        ElasticNet.solve = solve
        ElasticNet.update_weights = update_weights
        ElasticNet.update_kappa = update_kappa
        ElasticNet.update_intermediate_cities = update_intermediate_cities
        ElasticNet.D = D
        ElasticNet.L = L
        ElasticNet.get_city_to_intermediate_city_map = get_city_to_intermediate_city_map
        ElasticNet.get_tour = get_tour

        # Load city coordinates
        cities, norm_cities = load_cities(self.CONFIG["INPUT"])
        # Set N to number of cities
        self.N = cities.shape[0]
        # Calculate number of intermediate points between the cities
        self.M = int(self.CONFIG["M_FACTOR"] * self.N)
        # Initialize kappa to the starting value
        self.kappa = self.CONFIG["KAPPA_START"]
        # Set original_cities to the cities
        self.original_cities = cities
        # Set cities to the normalized cities
        self.cities = norm_cities
        # Generate intermediate cities around the centroid
        self.init_intermediate_cities(strategy="centroid")
        # A queue to hold the last two worst distances (for the stopping criterion)
        self.worst_dists = deque(maxlen=2)

In [117]:
def solve():
    with wandb.init() as run:
        distances = []
        worst_dists = []
        for _ in range(5):
            net = ElasticNet(
                init_intermediate_cities=init_intermediate_cities,
                solve=solve,
                update_weights=update_weights,
                update_kappa=update_kappa,
                update_intermediate_cities=update_intermediate_cities,
                D=D,
                L=L,
                get_city_to_intermediate_city_map=get_city_to_intermediate_city_map,
                get_tour=get_tour
            )
            distance, tour = net.solve()
            distances.append(distance)
            worst_dists.append(net.worst_dist)
        wandb.log({
            "distances": distances,
            "worst_dists": worst_dists,
            "median_worst_dist": np.median(worst_dists),
            "median_distance": np.median(distances),
        })

In [ ]:
wandb.agent(sweep_id, function=solve, count=1)

---------------

In [ ]:
net.worst_dist

0.041326854649664506

In [ ]:
net.weights[:5, :5]

array([[0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        3.90225047e-315, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        1.13822077e-146, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 9.99999921e-001, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 1.77015861e-009, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000]])

In [ ]:
net.intermediate_cities[:3]

array([[-0.22236826, -0.78673599],
       [ 0.05938106,  0.87667622],
       [-0.27894467, -0.82936032]])

------------

In [ ]:
import matplotlib.pyplot as plt